<a href="https://colab.research.google.com/github/ymoslem/Adaptive-MT-LLM-Fine-tuning/blob/main/Convert-Mistral-Finetuned-CTranslate2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert a fine-tuned model from Transformers to CTranslate2
This notebook is part of the repository [Adaptive-MT-LLM-Fine-tuning](https://github.com/ymoslem/Adaptive-MT-LLM-Fine-tuning).

# Load a model fine-tuned with PEFT

In [ ]:
!pip3 install transformers peft accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.0 MB/s eta 0:00:00


In [ ]:
!ls -lhat /content/drive/MyDrive/data/spanish/mistral_finetuning*

/content/drive/MyDrive/data/spanish/mistral_finetuning_v2:
total 20K
drwx------ 2 root root 4.0K Nov 26 02:12 checkpoint-608
drwx------ 2 root root 4.0K Nov 26 01:57 checkpoint-456
drwx------ 2 root root 4.0K Nov 26 01:41 checkpoint-304
drwx------ 2 root root 4.0K Nov 26 01:25 checkpoint-152
drwx------ 2 root root 4.0K Nov 26 01:09 runs

/content/drive/MyDrive/data/spanish/mistral_finetuning:
total 8.0K
drwx------ 2 root root 4.0K Nov 25 01:01 checkpoint-121
drwx------ 2 root root 4.0K Nov 25 00:44 runs


In [ ]:
import torch
from peft import PeftModel, PeftConfig
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# Miatral one epoch
adapter_path = "/content/drive/MyDrive/data/spanish/mistral_finetuning/checkpoint-121"

# Mistral 4 epochs
# adapter_path = "/content/drive/MyDrive/data/spanish/mistral_finetuning_v2/checkpoint-608"

cache_dir = "/content/drive/Shareddrives/adapt-yasmin/models/"
# offload_folder = ""

peft_model_id = adapter_path
peftconfig = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(peftconfig.base_model_name_or_path,
                                             device_map = "auto",
                                             # offload_folder = offload_folder,
                                             cache_dir = cache_dir
                                            )

tokenizer = AutoTokenizer.from_pretrained(peftconfig.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id)

print("Peft model loaded")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Peft model loaded


In [ ]:
import os
from google.colab import userdata

shared_drive = userdata.get("shared_drive")

final_model_path = os.path.join(shared_drive, "models/Mistral-7B-finetuned--v2-4epochs-26Nov")

merged_model = model.merge_and_unload()

merged_model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

# Convert to CTranslate2

In [ ]:
!pip3 install ctranslate2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 28.0 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

shared_drive = userdata.get("shared_drive")

directory = os.path.join(shared_drive, "models")

os.chdir(directory)
os.getcwd()

In [ ]:
ct2_model_dir = "ct2-mistral-finetuned-v1-25Nov"  # 1 epoch
# ct2_model_dir = "ct2-mistral-finetuned-v2-26Nov"  # 4 epochs

In [ ]:
!ct2-transformers-converter --model $final_model_path --quantization int8 --output_dir $ct2_model_dir

Loading checkpoint shards: 100% 6/6 [01:29<00:00, 14.95s/it]
